In [158]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import json
from tqdm import tqdm

In [159]:
options = Options()
options.headless = True
driver = webdriver.Firefox("./", options=options)

In [ ]:
driver.get("https://dona.pwr.edu.pl/szukaj/")

In [162]:
wait = WebDriverWait(driver, 20)

In [163]:
wait.until(EC.visibility_of_element_located((By.XPATH, "//span[.='Formularz jednostki']")))
driver.find_element_by_xpath("//span[.='Formularz jednostki']").click()

In [164]:
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#RadDock_formularze_C_RadButton_szukaj_zaawansowane")))
driver.find_element_by_css_selector("#RadDock_formularze_C_RadButton_szukaj_zaawansowane").click()

In [165]:
wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#RadComboBox_rok_zal_od_Input")))
driver.find_element_by_css_selector("#RadComboBox_rok_zal_od_Input").clear()
driver.find_element_by_css_selector("#RadComboBox_rok_zal_od_Input").send_keys("2010")
wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#RadComboBox_rok_zal_od_Input")))
driver.find_element_by_css_selector("#RadComboBox_rok_zal_od_Input").clear()
driver.find_element_by_css_selector("#RadComboBox_rok_zal_do_Input").send_keys("2020")

In [166]:
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#RadButton_filtry_zastosuj > span:nth-child(1)")))
driver.find_element_by_css_selector("#RadButton_filtry_zastosuj > span:nth-child(1)").click()

In [168]:
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#RadGrid_glowny_ctl00_ctl03_ctl01_PageSizeDropDownList")))
driver.find_element_by_css_selector("#RadGrid_glowny_ctl00_ctl03_ctl01_PageSizeDropDownList").click()
driver.find_element_by_css_selector("#RadGrid_glowny_ctl00_ctl03_ctl01_PageSizeDropDownList_DropDown").find_element_by_xpath("//li[.='50']").click()

In [169]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadAjaxLoadingPanel1RadGrid_glowny")))
WebDriverWait(driver, 30).until_not(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadAjaxLoadingPanel1RadGrid_glowny")))

True

In [171]:
driver.execute_script("__doPostBack('RadGrid_glowny$ctl00$ctl02$ctl00$ctl102','')");

In [157]:
num_pages = int(driver.find_element_by_css_selector(".rgInfoPart").find_elements_by_tag_name("strong")[1].text)

with tqdm(total=num_pages) as pbar:
    while True:
        rows = driver.find_element_by_css_selector("#RadGrid_glowny_ctl00 > tbody:nth-child(4)").find_elements_by_xpath("./tr")
        page_info = []
        for row in rows:
            entry = {}

            entry['authors'] = []
            authors = row.find_elements_by_xpath('.//span[@class="css_autor"]/..')
            for author in authors:
                entry['authors'].append({
                    'link': author.get_attribute('href'),
                    'name': author.find_element_by_xpath(".//span").text
                })

            entry['type'] = row.find_element_by_xpath(".//span[@class='RadLabel RadLabel_Bootstrap']").text
            entry['department'] = row.find_element_by_xpath(".//span[contains(@id, '_jednostka')]/span").text
            entry['year'] = row.find_element_by_xpath("./td[40]").text
            entry['text'] = "\n".join(row.find_element_by_xpath(".//span[contains(@id, '_opis_pracy_grid')]").text.split("\n")[1:])

            page_info.append(entry)

        page_num = driver.find_element_by_css_selector(".rgInfoPart").find_element_by_tag_name("strong").text

        with open(f'../data/raw_dona/{page_num}.json', 'w') as fp:
            json.dump(page_info, fp, indent=1, ensure_ascii=False)
            
        pbar.update(1)

        if int(page_num) < num_pages:
            driver.find_element_by_class_name("rgPageNext").click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadAjaxLoadingPanel1RadGrid_glowny")))
            WebDriverWait(driver, 30).until_not(EC.presence_of_element_located((By.CSS_SELECTOR, "#RadAjaxLoadingPanel1RadGrid_glowny")))
        else:
            break


  0%|          | 1/1017 [00:04<1:15:51,  4.48s/it]Parsed page 1 of 1017

  0%|          | 2/1017 [00:19<2:08:09,  7.58s/it]Parsed page 2 of 1017

  0%|          | 3/1017 [00:35<2:50:55, 10.11s/it]Parsed page 3 of 1017

  0%|          | 4/1017 [00:51<3:20:14, 11.86s/it]Parsed page 4 of 1017

  0%|          | 5/1017 [01:07<3:44:12, 13.29s/it]Parsed page 5 of 1017

  1%|          | 6/1017 [01:24<4:01:06, 14.31s/it]Parsed page 6 of 1017

  1%|          | 7/1017 [01:40<4:08:35, 14.77s/it]Parsed page 7 of 1017

  1%|          | 8/1017 [01:55<4:12:11, 15.00s/it]Parsed page 8 of 1017

  1%|          | 9/1017 [02:13<4:23:22, 15.68s/it]Parsed page 9 of 1017

  1%|          | 10/1017 [02:31<4:34:21, 16.35s/it]Parsed page 10 of 1017

  1%|          | 11/1017 [02:48<4:41:41, 16.80s/it]Parsed page 11 of 1017

  1%|          | 12/1017 [03:07<4:50:09, 17.32s/it]Parsed page 12 of 1017

  1%|▏         | 13/1017 [03:28<5:07:04, 18.35s/it]Parsed page 13 of 1017

  1%|▏         | 14/1017 [03:46<5:08:29, 1

KeyboardInterrupt: 